<a href="https://colab.research.google.com/github/skyjadel/chang-s_titanic/blob/main/Titanic_Task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [38]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_csv('https://raw.githubusercontent.com/dsindy/kaggle-titanic/master/data/train.csv')

In [2]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [12]:
df = df[list(set(df.columns).difference(['Ticket','Name','Cabin']))]
df

,PassengerId,Sex,Survived,Parch,Fare,Age,Pclass,SibSp,Embarked
0,1,male,0,0,7.2500,22.0,3,1,S
1,2,female,1,0,71.2833,38.0,1,1,C
2,3,female,1,0,7.9250,26.0,3,0,S
3,4,female,1,0,53.1000,35.0,1,1,S
4,5,male,0,0,8.0500,35.0,3,0,S
...,...,...,...,...,...,...,...,...,...
886,887,male,0,0,13.0000,27.0,2,0,S
887,888,female,1,0,30.0000,19.0,1,0,S
888,889,female,0,2,23.4500,NaN,3,1,S
889,890,male,1,0,30.0000,26.0,1,0,C


# 資料前處理

## 區分 Training Set 與 Testing Set

In [13]:
train_df, test_df = train_test_split(df, test_size=0.2)
print(f'{len(train_df)} rows in training set.')
print(f'{len(test_df)} rows in testing set.')

712 rows in training set.
179 rows in testing set.


## 補缺失值

從上次的EDA我們知道，有缺失值的欄位有 Age 和 Embarked，Age將補上training set 的平均值，Embarked將補上trainin set 的最多分類。

In [25]:
# 找出哪個港口上船的人最多
grouped = train_df.groupby('Embarked')
harbor_counts = {g[0]:len(g[1]) for g in grouped}

max_count = 0
for k, v in harbor_counts.items():
  if v > max_count:
    max_count = v
    major_embarked_harbor = k

In [29]:
# Training Set 的年齡平均值
mean_age_train = train_df['Age'].mean()

In [41]:
#因為對訓練集與測試集的前處理要一模一樣，所以寫成函式
def fill_nan(df, age_value=mean_age_train, embarked_value=major_embarked_harbor):
  df = df.fillna({'Age':age_value, 'Embarked':embarked_value})
  return df

## Labeling

Sex 欄位改稱 Sex_female 欄位 <br>
Embarked 使用onehot

In [68]:
def labeling(df):
  df = pd.concat([df, pd.get_dummies(df[['Sex', 'Embarked']])], axis=1)
  df.drop(['Sex', 'Embarked', 'Sex_male'], axis=1, inplace=True)
  return df

## 前處理函式

In [71]:
def preprocess(df):
  return labeling(fill_nan(df))

In [72]:
train_df = preprocess(train_df)
test_df = preprocess(test_df)

In [73]:
train_df

,PassengerId,Survived,Parch,Fare,Age,Pclass,SibSp,Sex_female,Embarked_C,Embarked_Q,Embarked_S
448,449,1,1,19.2583,5.000000,3,2,True,True,False,False
85,86,1,0,15.8500,33.000000,3,3,True,False,False,True
164,165,0,1,39.6875,1.000000,3,4,False,False,False,True
138,139,0,0,9.2167,16.000000,3,0,False,False,False,True
349,350,0,0,8.6625,42.000000,3,0,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...
434,435,0,0,55.9000,50.000000,1,1,False,False,False,True
578,579,0,0,14.4583,29.567418,3,1,True,True,False,False
634,635,0,2,27.9000,9.000000,3,3,True,False,False,True
435,436,1,2,120.0000,14.000000,1,1,True,False,False,True


# 訓練

In [88]:
columns_X = list(set(train_df.columns) - {'Survived', 'PassengerId'})
columns_y = ['Survived']

train_X = train_df[columns_X]
train_y = train_df[columns_y]
test_X = test_df[columns_X]
test_y = test_df[columns_y]

In [84]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score


log = LogisticRegression(random_state=0, max_iter=3000)
scores = cross_val_score(log, train_X, train_y.values.ravel(),cv=5,scoring='accuracy')
print(scores)
print(f'平均準確率{np.mean(scores):.4f}')

[0.79020979 0.78321678 0.76760563 0.72535211 0.82394366]
平均準確率0.7781


In [98]:
log.fit(np.array(train_X), np.array(train_y).reshape(-1))

LogisticRegression(max_iter=3000, random_state=0)

In [99]:
pred_y = log.predict(np.array(test_X))
true_y = yt = np.array(test_y).reshape(-1)
acc_samples = sum(true_y * pred_y) + sum((1-true_y)*(1-pred_y))
print(f'Accuracy for testing set is {acc_samples/len(test_df):.4f} ({acc_samples}/{len(test_df)})')

Accuracy for testing set is 0.8156 (146/179)
